To verify some theories about dynamic envs

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import product

In [2]:
#v1 = lambda w: w >= 2
#v2 = lambda w: 2 * (w >= 4)

def example(w):
    if w >= 3:
        return 3
    if w >= 2:
        return 2
    if w >= 1:
        return 1
    return 0


In [3]:
#x = np.linspace(0, 10, 100)
#plt.plot(x, v1(x))
#plt.plot(x, v2(x));

In [4]:
def div_rule(alpha):
    # hard-code to illustrate the example!
    if len(alpha) == 2:
        if alpha[0] > alpha[1]:
            return [1.0, 0.0]
        elif alpha[0] < alpha[1]:
            return [0.0, 1.0]
        
    return [alphai/sum(alpha) for alphai in alpha]

In [5]:
def evaluate(path, Ws, envs, f):
    '''
    path: assignment
    envs: list of function 
    assume everyone start out with M=0
    
    division rule is assume to be Mi/MC
    f is a division_rule
    '''
    n = len(Ws)
    T = len(path)
    M = [[1 for _ in range(n)] for _ in range(T+1)]
    for t, assignment in enumerate(path, start=1):
        for room_num, room in enumerate(assignment):
            v = envs[room_num]
            W = sum([Ws[player] for player in room])
            alpha = [M[t-1][player] for player in room]
            div = f(alpha)
            for playerId, player in enumerate(room):
                M[t][player] = M[t-1][player] + v(W) * div[playerId]
    
    return M

In [6]:
# assignment1 = [[0], [1,2]]
# assignment2 = [[1], [0,2]]
# path = [assignment1, assignment2]
# envs = [v1, v2]
# m = len(envs)

In [7]:
# x = 2
# Ws = [1,x,3]
# n = len(Ws)

# Example 1: increase weight in fixed path doesn't help

The following is an example with fixed $\omega$. Increase the weight of player 1 from $1.5$ to $1.6$ actually makes $\{M_i \geq M_j\}$ no longer true.

In [8]:
x = 1.6 # x = [1.5, 1.6]
Ws = [x,1.4,1]
n = len(Ws)
i = 0
j = 1
h = 2

In [9]:
# counter-example!
assignment1 = [[i, j], [h]]
assignment2 = [[i], [j, h]]
path = [assignment1, assignment2]
envs = [example, example]
m = len(envs)

In [10]:
'''
A path is a t-length list of outcome where t = length of the trajectory.
An outcome is a m-list, where m is the number of environments
Each outcome[room] is the list of players in that room
'''

'\nA path is a t-length list of outcome where t = length of the trajectory.\nAn outcome is a m-list, where m is the number of environments\nEach outcome[room] is the list of players in that room\n'

In [11]:
path

[[[0, 1], [2]], [[0], [1, 2]]]

In [12]:
evaluate(path, Ws, envs, div_rule)

[[1, 1, 1], [2.5, 2.5, 2.0], [3.5, 4.5, 2.0]]

In [13]:
def generate_all_functions(X, Y):
    # generate all functions from X to Y
    # https://stackoverflow.com/questions/28649683/a-list-of-all-functions-from-x-to-y-in-python
    return [dict(zip(X,y)) for y in product(Y,repeat=len(X))] 

In [14]:
fns = generate_all_functions(range(n), range(len(envs)))

In [15]:
def generate_all_outcomes(assignment_dict, m):
    ret = [[] for _ in range(m)]
    for player, room in assignment_dict.items():
        ret[room].append(player)
    return ret

In [16]:
outcomes = [generate_all_outcomes(d, m) for d in fns]
outcomes

[[[0, 1, 2], []],
 [[0, 1], [2]],
 [[0, 2], [1]],
 [[0], [1, 2]],
 [[1, 2], [0]],
 [[1], [0, 2]],
 [[2], [0, 1]],
 [[], [0, 1, 2]]]

In [17]:
def generate_all_paths(outcomes, T):
    # T = length of outcome
    return list(product(outcomes,repeat=T))

In [18]:
T = 2
paths = generate_all_paths(outcomes, T)
len(paths)

64

In [19]:
expected_loss = 0
for path in paths:
    ret = evaluate(path, Ws, envs, div_rule)[-1]
    print('path:', path, 'ret:', ret)
    if ret[i] >= ret[j]:
        expected_loss += 1

path: ([[0, 1, 2], []], [[0, 1, 2], []]) ret: [3.0, 3.0, 3.0]
path: ([[0, 1, 2], []], [[0, 1], [2]]) ret: [3.5, 3.5, 3.0]
path: ([[0, 1, 2], []], [[0, 2], [1]]) ret: [3.0, 3.0, 3.0]
path: ([[0, 1, 2], []], [[0], [1, 2]]) ret: [3.0, 3.0, 3.0]
path: ([[0, 1, 2], []], [[1, 2], [0]]) ret: [3.0, 3.0, 3.0]
path: ([[0, 1, 2], []], [[1], [0, 2]]) ret: [3.0, 3.0, 3.0]
path: ([[0, 1, 2], []], [[2], [0, 1]]) ret: [3.5, 3.5, 3.0]
path: ([[0, 1, 2], []], [[], [0, 1, 2]]) ret: [3.0, 3.0, 3.0]
path: ([[0, 1], [2]], [[0, 1, 2], []]) ret: [3.571428571428571, 3.571428571428571, 2.857142857142857]
path: ([[0, 1], [2]], [[0, 1], [2]]) ret: [4.0, 4.0, 3.0]
path: ([[0, 1], [2]], [[0, 2], [1]]) ret: [4.5, 3.5, 2.0]
path: ([[0, 1], [2]], [[0], [1, 2]]) ret: [3.5, 4.5, 2.0]
path: ([[0, 1], [2]], [[1, 2], [0]]) ret: [3.5, 4.5, 2.0]
path: ([[0, 1], [2]], [[1], [0, 2]]) ret: [4.5, 3.5, 2.0]
path: ([[0, 1], [2]], [[2], [0, 1]]) ret: [4.0, 4.0, 3.0]
path: ([[0, 1], [2]], [[], [0, 1, 2]]) ret: [3.571428571428571, 3.

In [20]:
expected_loss/len(paths)

0.9375

# Example 2: swapping doesn't help

In [21]:
i = 0
j = 1
k = 2
h = 3

In [22]:
def example2(w):
    if w >= 3:
        return 3.2
    if w >= 2:
        return 2
    if w >= 1:
        return 1
    return 0

In [23]:
# wi < wj but Mi > Mj
Ws = [1,1.5,1.5,1]
#Ws = [1,1.5,1.5,1.5]
#Ws = [1,1.2,1.4,1]
#Ws = [1.1,1.2,1.4,1]

assignment1 = [[i, k], [j], [h]]
assignment2 = [[i], [j], [k, h]]
assignment3 = [[i, k, h], [j], []]

path = [assignment1, assignment2, assignment3]
envs = [example2, example2, example2]

In [24]:
def div_rule2(alpha):
    # hard-code to illustrate the example!
    if len(alpha) == 3:
        m = max(alpha)
        if len([x for x in alpha if x == m]) == 1: # have no more than 1 max
            for i, x in enumerate(alpha):
                if x != m: alpha[i] = 0
    return [alphai/sum(alpha) for alphai in alpha]

In [25]:
evaluate(path, Ws, envs, div_rule2)

[[1, 1, 1, 1],
 [2.0, 2.0, 2.0, 2.0],
 [3.0, 3.0, 3.0, 3.0],
 [4.066666666666666, 4.0, 4.066666666666666, 4.066666666666666]]

In [26]:
# swap i and j
# Mi still > Mj ==> This swapping cannot work either!
assignment1 = [[j, k], [i], [h]]
assignment2 = [[j], [i], [k, h]]
assignment3 = [[j, k, h], [i], []]

In [27]:
path = [assignment1, assignment2, assignment3]
evaluate(path, Ws, envs, div_rule2)

[[1, 1, 1, 1],
 [2.0, 2.6, 2.6, 2.0],
 [3.0, 3.6, 3.730434782608696, 2.8695652173913047],
 [4.0, 3.6, 6.930434782608696, 2.8695652173913047]]

In [28]:
n = len(Ws)
m = 3
fns = generate_all_functions(range(n), range(len(envs)))
outcomes = [generate_all_outcomes(d, m) for d in fns]

T = 3
paths = generate_all_paths(outcomes, T)

rets = []
expected_loss = 0
for path in paths:
    ret = evaluate(path, Ws, envs, div_rule)[-1]
    #print('path:', path, 'ret:', ret)
    if ret[j] >= ret[i]:
        expected_loss += 1
    rets.append(ret)

In [29]:
expected_loss/len(paths)

0.7895645988924452

In [30]:
expected_loss = 0
for ret in rets:
    if ret[h] >= ret[i]:
        expected_loss += 1

In [31]:
expected_loss/len(rets)

0.7261596301376823

## Validate C++ Example

In [32]:
Ws = [1,2,3]
i = 0
j = 2

In [33]:
def env(w):
    if w >= 2:
        return 3
    if w >= 1:
        return 1
    return 0

In [34]:
def div_rule_C(alpha):
    return [x/sum(alpha) for x in alpha]

In [35]:
envs = [env, env]
T=1

In [36]:
n = len(Ws)
m = len(envs)
fns = generate_all_functions(range(n), range(len(envs)))
outcomes = [generate_all_outcomes(d, m) for d in fns]

paths = generate_all_paths(outcomes, T)

rets = []
expected_loss = 0
for path in paths:
    print('path: ', path)
    ret = evaluate(path, Ws, envs, div_rule_C)[-1]
    #print('path:', path, 'ret:', ret)
    if ret[j] >= ret[i]:
        expected_loss += 1
    print(ret)
    rets.append(ret)

path:  ([[0, 1, 2], []], [[0, 1, 2], []])
[3.0, 3.0, 3.0]
path:  ([[0, 1, 2], []], [[0, 1], [2]])
[3.5, 3.5, 5.0]
path:  ([[0, 1, 2], []], [[0, 2], [1]])
[3.5, 5.0, 3.5]
path:  ([[0, 1, 2], []], [[0], [1, 2]])
[3.0, 3.5, 3.5]
path:  ([[0, 1, 2], []], [[1, 2], [0]])
[3.0, 3.5, 3.5]
path:  ([[0, 1, 2], []], [[1], [0, 2]])
[3.5, 5.0, 3.5]
path:  ([[0, 1, 2], []], [[2], [0, 1]])
[3.5, 3.5, 5.0]
path:  ([[0, 1, 2], []], [[], [0, 1, 2]])
[3.0, 3.0, 3.0]
path:  ([[0, 1], [2]], [[0, 1, 2], []])
[3.3333333333333335, 3.3333333333333335, 5.333333333333333]
path:  ([[0, 1], [2]], [[0, 1], [2]])
[4.0, 4.0, 7.0]
path:  ([[0, 1], [2]], [[0, 2], [1]])
[3.653846153846154, 5.5, 5.846153846153847]
path:  ([[0, 1], [2]], [[0], [1, 2]])
[3.5, 3.653846153846154, 5.846153846153847]
path:  ([[0, 1], [2]], [[1, 2], [0]])
[3.5, 3.653846153846154, 5.846153846153847]
path:  ([[0, 1], [2]], [[1], [0, 2]])
[3.653846153846154, 5.5, 5.846153846153847]
path:  ([[0, 1], [2]], [[2], [0, 1]])
[4.0, 4.0, 7.0]
path:  ([[0,

In [37]:
expected_loss

64

In [38]:
len(paths)

64